In [15]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

In [16]:
df = pd.read_csv('processed_data.csv')
display(df.head(50))

,label,subject,email_to,email_from,message
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...
5,1,Which is duty,manager@speedy.uwaterloo.ca,"""Bobby L. Fleming"" <zvyrepeated@liselebel.com>",Content-Type: multipart/alternative;\n ...
6,1,For Theorize,<theorize@plg.uwaterloo.ca>,"""B.J. Badisang"" <mitigatemicroprocessor@g-rich...","Content-Type: text/plain;\n\tcharset=""Windows-..."
7,1,Theorize get inside for local esc0rts who do i...,"""Theorize"" <theorize@plg.uwaterloo.ca>","""Esteban Everett"" <dbmagyrln@dbmag.com>",Content-Type: Multipart/Alternative;\n bounda...
8,1,Losing Weight Quickly,theorize@plg.uwaterloo.ca,Real Loss <fibond@terra.com.do>,"Content-Type: text/plain;\n charset=""iso-88..."
9,0,[R] Confidence-Intervals.... help...,r-help@stat.math.ethz.ch,"""Jochen.F"" <jjfahr@ucalgary.ca>",Hi...\n\nI have to use R to find out the 90% c...


In [17]:
df['importance'] = 0
df['urgency'] = 0
ham = df.loc[df['label'] == 0, ['importance', 'urgency', 'subject', 'message']].reset_index(drop=True)
ham = ham.dropna()
display(ham.head(50))

,importance,urgency,subject,message
0,0,0,Typo in /debian/README,"Hi, i've just updated from the gulus and I che..."
1,0,0,[R] Confidence-Intervals.... help...,Hi...\n\nI have to use R to find out the 90% c...
2,0,0,Re: [R] Confidence-Intervals.... help...,Hm... sounds like a homework problem to me...\...
3,0,0,Re: Broken ACPI with kernel 2.6.18-4-686 in HP...,Gruesse!\n* V.Harishankar schrieb am [08.04.07...
4,0,0,[R] Failure of mcsamp() but not mcmcsamp(),"Daer r-helpers,\n\nCan anyone help with the fo..."
5,0,0,Your daily e-mail from the BBC,"Sunday, 08 A=\npril, 2007, 18:00 GMT 14:00 -04..."
6,0,0,CNN Alerts: bush,Alert Name: bush Gingrich: Gonzales should con...
7,0,0,AccuWeather.com Alert(tm) Forecast For User|AV...,AccuWeather 7-Day Forecast for \nBEVERLY HILLS...
8,0,0,AccuWeather.com Alert(tm) Forecast For User|AV...,AccuWeather 7-Day Forecast for \nBEVERLY HILLS...
9,0,0,[perl #42349] [TODO] dotnet - fix various ops ...,# New Ticket Created by Paul Cochrane \n# Ple...


In [18]:
ham['message'] = ham['message'].apply(lambda x: x.replace('\n', ' '))
ham['subject'] = ham['subject'].apply(lambda x: x.replace('\n', ' '))
display(ham.head(100))
print(len(ham))

,importance,urgency,subject,message
0,0,0,Typo in /debian/README,"Hi, i've just updated from the gulus and I che..."
1,0,0,[R] Confidence-Intervals.... help...,Hi... I have to use R to find out the 90% con...
2,0,0,Re: [R] Confidence-Intervals.... help...,Hm... sounds like a homework problem to me... ...
3,0,0,Re: Broken ACPI with kernel 2.6.18-4-686 in HP...,Gruesse! * V.Harishankar schrieb am [08.04.07 ...
4,0,0,[R] Failure of mcsamp() but not mcmcsamp(),"Daer r-helpers, Can anyone help with the foll..."
...,...,...,...,...
95,0,0,[perl #42385] [TODO] imcc - remove global vari...,# New Ticket Created by Paul Cochrane # Plea...
96,0,0,=?iso-8859-1?Q?Today's_WeatherDirect_Forecast_...,subscribe | change profile | contact us Long T...
97,0,0,Officials: U.S. to take legal action against C...,"Apr 9, 2007 05:15 AM EDT Top Stories Weather ..."
98,0,0,Re: [R] R:Maximum likelihood estimation using ...,"Joey, First of all, it is bad habit to call y..."


25213


In [19]:
print(ham.columns.tolist())

['importance', 'urgency', 'subject', 'message']


In [20]:
words = set(nltk.corpus.words.words())
for i in range(0, 2000, 5):
    message = str(ham.loc[i, 'message'])
    ham.loc[i, 'message'] = " ".join(w for w in nltk.wordpunct_tokenize(message) if w.lower() in words)
display(ham.head(500))

,importance,urgency,subject,message
0,0,0,Typo in /debian/README,Hi i just from the and I check on other It the...
1,0,0,[R] Confidence-Intervals.... help...,Hi... I have to use R to find out the 90% con...
2,0,0,Re: [R] Confidence-Intervals.... help...,Hm... sounds like a homework problem to me... ...
3,0,0,Re: Broken ACPI with kernel 2.6.18-4-686 in HP...,Gruesse! * V.Harishankar schrieb am [08.04.07 ...
4,0,0,[R] Failure of mcsamp() but not mcmcsamp(),"Daer r-helpers, Can anyone help with the foll..."
...,...,...,...,...
495,0,0,The Internet Patrol - Blogger Freed from Ja...,Content Disposition Content Length Content Typ...
496,0,0,[cc-community] Wikitravel nominated for Webby ...,Content-Type: multipart/signed; micalg=sha1; \...
497,0,0,Re: Sorting dir output,Content-Type: multipart/alternative; boundary=...
498,0,0,tdb docs,I noticed the source/tdb/docs/README is a bit ...


In [21]:
ham = ham.drop(columns = ['message'])
ham = ham.sample(frac=1)
display(ham.head(50))
ham = ham.reset_index()
splits = list(np.linspace(0, 2000, dtype=int))

,importance,urgency,subject
14190,0,0,Re: [R] Legend outside plotting area
1336,0,0,CNN Alerts: bush
24647,0,0,svn commit: samba r23697 - in \tbranches/SAMBA...
22966,0,0,Nov-D: Article: Indonesian VP OKs Marijuana As...
18446,0,0,Stock Notification for SatCon Technology Corpo...
15342,0,0,Re: svn commit: samba r23284 - in \tbranches/S...
9409,0,0,[perl #42919] [BUG] and [PATCH] overriding 'in...
23292,0,0,=?iso-8859-1?Q?Today's_WeatherDirect_Forecast_...
18705,0,0,Re: [R] Trouble making JRI.jar with Ubuntu and...
18928,0,0,Re: [svn:perl6-synopsis] r14421 - doc/trunk/de...


In [11]:
splits.reset_index()

,index,importance,urgency,subject
0,10437,0,0,Re: [R] conversion into capital letter
1,649,0,0,svn commit: samba r22165 - in \tbranches/SAMBA...
2,4375,0,0,New SEC Document(s) for ONEOK Inc.
3,20696,0,0,Re: doubletalk lt
4,6800,0,0,[R] Warnings in package dependencies and /src ...
...,...,...,...,...
25208,7292,0,0,Stock Notification for SatCon Technology Corpo...
25209,23682,0,0,Re: How can i make a perl program lauch anothe...
25210,6240,0,0,Re: [SAMBA4] we should create just one test en...
25211,7982,0,0,Police officer dead in shooting
